In [4]:
import tensorflow as tf
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

In [8]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [9]:
X_train.dtype, y_train.dtype, X_test.dtype, y_test.dtype

(dtype('uint8'), dtype('uint8'), dtype('uint8'), dtype('uint8'))

In [181]:
def get_im2col_indices(X_shape, field_height, field_width, padding=1, stride=1):
    # First figure out what the size of the output should be
    # Input shape
    N, C, H, W = X_shape
    
    # Kernel shape
    # field_height, field_width = kernel_shape
    field_C = C
    
    # Output shape
    assert (H + (2 * padding) - field_height) % stride == 0
    assert (W + (2 * padding) - field_width) % stride == 0
    out_height = int(((H + (2 * padding) - field_height) / stride) + 1)
    out_width = int(((W + (2 * padding) - field_width) / stride) + 1)
    out_C = 1 # the output channel/ depth
    print('out_C, out_height, out_width', out_C, out_height, out_width)

    # Row, Height, i
    i0 = np.repeat(np.arange(field_height), field_width)
    print('i0, i0.shape', i0, i0.shape)
    i0 = np.tile(i0, field_C)
    print('i0, i0.shape', i0, i0.shape)
    i1 = np.repeat(np.arange(out_height), out_width)
    print('i1, i1.shape', i1, i1.shape)
    i1 = np.tile(i1, out_C)
    print('i1, i1.shape', i1, i1.shape)
    i1 *= stride
    print('i1, i1.shape', i1, i1.shape)
    
    # Column, Width, j
    j0 = np.tile(np.arange(field_width), field_height * field_C)
    print('j0, j0.shape', j0, j0.shape)
    j1 = np.tile(np.arange(out_width), out_height * out_C)
    print('j1, j1.shape', j1, j1.shape)
    j1 *= stride
    print('j1, j1.shape', j1, j1.shape)

    # Channel, Depth, K
    k0 = np.repeat(np.arange(field_C), field_height * field_width) #.reshape(-1, 1) # out_C = 1
    print('k0, k0.shape', k0, k0.shape)
    k1 = np.repeat(np.arange(out_C), out_height * out_width) #.reshape(-1, 1) # out_C = 1
    print('k1, k1.shape', k1, k1.shape)
    k1 *= stride
    print('k1, k1.shape', k1, k1.shape)
    
    # Indices: i, j, k index
    i = i0.reshape(-1, 1) + i1.reshape(1, -1)
    print('i0.reshape(-1, 1) , i1.reshape(1, -1), i', i0.reshape(-1, 1) , i1.reshape(1, -1), i)
    print('i0.reshape(-1, 1).shape , i1.reshape(1, -1).shape, i.shape', 
          i0.reshape(-1, 1).shape , i1.reshape(1, -1).shape, i.shape)
    j = j0.reshape(-1, 1) + j1.reshape(1, -1)
    print('j0.reshape(-1, 1) , j1.reshape(1, -1), j',
         j0.reshape(-1, 1) , j1.reshape(1, -1), j)
    print('j0.reshape(-1, 1).shape , j1.reshape(1, -1).shape, j.shape', 
         j0.reshape(-1, 1).shape , j1.reshape(1, -1).shape, j.shape)
    k = k0.reshape(-1, 1) + k1.reshape(1, -1)
    print('k0.reshape(-1, 1), k1.reshape(1, -1), k',
         k0.reshape(-1, 1), k1.reshape(1, -1), k)
    print('k0.reshape(-1, 1).shape , k1.reshape(1, -1).shape, k.shape',
         k0.reshape(-1, 1).shape , k1.reshape(1, -1).shape, k.shape)
    
    return (k.astype(int), i.astype(int), j.astype(int))
#     return i.astype(int)

In [182]:
np.repeat(np.arange(3), 2)

array([0, 0, 1, 1, 2, 2])

In [183]:
np.repeat(np.arange(2), 5)

array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])

In [184]:
np.tile(np.arange(3), 4)

array([0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2])

In [185]:
np.repeat(np.arange(3), 4)

array([0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2])

In [186]:
np.tile(np.repeat(np.arange(2), 4), 2)

array([0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1])

In [187]:
img_shape = (1, 28, 28)
img_shape
# X_train = X_train.reshape([-1, *img_shape]) # (-1, img_shape[0], img_shape[1], img_shape[2])
# X_val = X_val.reshape(-1, *img_shape)
# X_test = X_test.reshape(-1, *img_shape)
# X_train.shape, X_val.shape, X_test.shape
# # X_train[0, :10, :10, :10]
# M, D, C

(1, 28, 28)

In [188]:
Xtrain = X_train.reshape(-1, *img_shape)

In [189]:
import numpy as np
k, i, j = get_im2col_indices(X_shape=Xtrain.shape, field_height=3, field_width=3, padding=1, stride=1)

out_C, out_height, out_width 1 28 28
i0, i0.shape [0 0 0 1 1 1 2 2 2] (9,)
i0, i0.shape [0 0 0 1 1 1 2 2 2] (9,)
i1, i1.shape [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  2  2  2  2  2  3  3  3  3  3  3  3  3  3  3  3  3
  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  4  4  4  4  4  4  4  4
  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  5  5  5  5
  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5
  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
  6  6  6  6  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7
  7  7  7  7  7  7  7  7  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8  8  8  8  8  8  8  8  8  8  8  9  9  9  9  9  9  9  9  9  9  9  9
  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9 10 10 10 10 10 10 10 1

In [190]:
k.shape

(9, 784)

In [191]:
j.shape

(9, 784)

In [192]:
i.shape

(9, 784)

In [193]:
k.dtype

dtype('int64')

In [194]:
i.dtype, j.dtype

(dtype('int64'), dtype('int64'))

In [195]:
k

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [196]:
i

array([[ 0,  0,  0, ..., 27, 27, 27],
       [ 0,  0,  0, ..., 27, 27, 27],
       [ 0,  0,  0, ..., 27, 27, 27],
       ...,
       [ 2,  2,  2, ..., 29, 29, 29],
       [ 2,  2,  2, ..., 29, 29, 29],
       [ 2,  2,  2, ..., 29, 29, 29]])

In [197]:
j

array([[ 0,  1,  2, ..., 25, 26, 27],
       [ 1,  2,  3, ..., 26, 27, 28],
       [ 2,  3,  4, ..., 27, 28, 29],
       ...,
       [ 0,  1,  2, ..., 25, 26, 27],
       [ 1,  2,  3, ..., 26, 27, 28],
       [ 2,  3,  4, ..., 27, 28, 29]])